<a href="https://colab.research.google.com/github/Muraka64/IOTs_with_python/blob/main/Ch_%EB%B6%80%EB%A1%9D_1.%20%EC%96%91%EB%B9%84%EB%A1%80%EC%A1%B0%EC%A0%95%EB%B2%95(RAS)_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

In [31]:
np.set_printoptions(precision=3,suppress=True) # 결과를 소수 3자리까지만 보여주는 옵션

In [32]:
url_Tt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Total_Matrix_3_secs_1_va.csv"  # 기준연도 거래표
Tot_19 = pd.read_csv(url_Tt_3)
Tot_19_num = Tot_19.to_numpy()
Tot_19

,Unnamed: 0,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
0,농산품,4.0,38.0,11.0,53.0,18.0,0.0,1.0,19.0,72.0
1,공산품,16.0,944.0,401.0,1361.0,213.0,167.0,628.0,1008.0,2369.0
2,서비스,7.0,257.0,693.0,957.0,1032.0,432.0,131.0,1595.0,2552.0
3,중간투입계,27.0,1239.0,1105.0,2371.0,1263.0,599.0,760.0,2622.0,4993.0
4,부가가치계,33.0,553.0,1342.0,1928.0,0.0,0.0,0.0,0.0,0.0
5,총투입액,60.0,1792.0,2447.0,4299.0,0.0,0.0,0.0,0.0,0.0


In [33]:
url_Tt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Estimated_table.csv"  # 비교연도 거래표
Tot_est = pd.read_csv(url_Tt_3)
Tot_Tr_num = Tot_est.to_numpy()
Tot_est

,구분,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
0,농산품,0,0,0,60,19,0,1,20,80
1,공산품,0,0,0,1513,215,170,635,1020,2533
2,서비스,0,0,0,1106,1035,455,130,1620,2726
3,중간투입계,30,1484,1165,2679,1269,625,766,2660,5339
4,부가가치계,35,568,1355,1958,0,0,0,0,0
5,총투입액,65,2052,2520,4637,0,0,0,0,0


In [34]:
Tb = Tot_19_num[0:7,1:10]
Tr = Tot_Tr_num[0:7,1:10]
Tr

array([[0, 0, 0, 60, 19, 0, 1, 20, 80],
       [0, 0, 0, 1513, 215, 170, 635, 1020, 2533],
       [0, 0, 0, 1106, 1035, 455, 130, 1620, 2726],
       [30, 1484, 1165, 2679, 1269, 625, 766, 2660, 5339],
       [35, 568, 1355, 1958, 0, 0, 0, 0, 0],
       [65, 2052, 2520, 4637, 0, 0, 0, 0, 0]], dtype=object)

In [35]:
Ct = Tb[-1,0:3]  # 산출액
Ct_m = np.tile(Ct, reps=[3,1]) # 산출액을 3번 행반복
Ct_m

array([[60.0, 1792.0, 2447.0],
       [60.0, 1792.0, 2447.0],
       [60.0, 1792.0, 2447.0]], dtype=object)

In [36]:
Ct_ref = Tr[5,0:3]  # 비교연도 산출액
Ct_ref


array([65, 2052, 2520], dtype=object)

In [37]:
At=Tb[0:3,0:3]/Ct_m # 기준연도 중간투입계수
Row_vec = Tr[0:3,4] # 중간수요계
Col_vec = Tr[3, 1:4] # 중간투입계

er = 1
ec = 1

In [38]:
# while 반복문으로 RAS 실행

i=0
while True:
  if (er<0.0000001) and (ec<0.0000001):
    break
  M = At@np.diag(Ct_ref)
  R_sm = M.sum(axis=1)
  R = (Row_vec/R_sm) #행수정계수
  print('행수정계수_R', '\n', R)
  At = np.diag(R)@At

  M = At@np.diag(Ct_ref)
  C_sm = M.sum(axis=0)
  S = (Col_vec.T/C_sm) #열수정계수
  At = At@np.diag(S)

  print('열수정계수_S', '\n', S)
  er = abs(R-1).sum()
  es = abs(S-1).sum()

  i = i+1
  if er<0.0000001 or ec<0.0000001:
    break
  print('행수정계수 절대합_ER', er)
  print('열수정계수 절대합_EC', es, '\n')
  print('반복회수', i)

print('\n', '총 반복회수', '\n', i)


print('\n', '최종투입계수','\n', At, '\n')
Mf = np.array(At@np.diag(Ct_ref), dtype=float).round(1)

print('\n', '추정된 최종거래표', '\n', Mf)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
행수정계수 절대합_ER 2.285472972972973
열수정계수 절대합_EC 9.595744680851062 

반복회수 1145
행수정계수_R 
 [0.23817567567567566 0.23817567567567566 0.23817567567567569]
열수정계수_S 
 [4.198581560283688 4.198581560283688 4.198581560283688]
행수정계수 절대합_ER 2.285472972972973
열수정계수 절대합_EC 9.595744680851062 

반복회수 1146
행수정계수_R 
 [0.2381756756756757 0.23817567567567574 0.23817567567567569]
열수정계수_S 
 [4.198581560283688 4.198581560283688 4.198581560283688]
행수정계수 절대합_ER 2.285472972972973
열수정계수 절대합_EC 9.595744680851062 

반복회수 1147
행수정계수_R 
 [0.23817567567567566 0.23817567567567566 0.23817567567567569]
열수정계수_S 
 [4.198581560283688 4.198581560283688 4.198581560283688]
행수정계수 절대합_ER 2.285472972972973
열수정계수 절대합_EC 9.595744680851062 

반복회수 1148
행수정계수_R 
 [0.2381756756756757 0.23817567567567574 0.23817567567567569]
열수정계수_S 
 [4.198581560283688 4.198581560283688 4.198581560283688]
행수정계수 절대합_ER 2.285472972972973
열수정계수 절대합_EC 9.595744680851062 

반복회수 1149
행수정계수_R 
 [0.23817567567567566 0.23817567567

KeyboardInterrupt: 

In [ ]:
df_Mf = pd.DataFrame(Mf)
df_Mf.to_csv('RAS결과.csv')